In [ ]:
class HybridRecommender:
    def __init__(self, movies_df, ratings_df):
        self.movies_df = movies_df
        self.ratings_df = ratings_df
        self.movie_features = None
        self.user_ratings = None
        self.user_predictions = None
        self.movie_predictions = None
        self.hybrid_predictions = None

    def preprocess_data(self):
        self.movie_features = self.movies_df['genres'].str.get_dummies(sep='|')
        self.user_ratings = self.ratings_df.pivot(index='userId', columns='movieId', values='rating')

    def collaborative_filtering(self):
        similar_users = cosine_similarity(self.user_ratings)
        self.user_predictions = pd.DataFrame(similar_users.dot(self.user_ratings.fillna(0)), index=self.user_ratings.index, columns=self.user_ratings.columns)

    def content_filtering(self):
        cv = CountVectorizer()
        movie_matrix = cv.fit_transform(self.movies_df['genres'])
        movie_similarities = cosine_similarity(movie_matrix)
        self.movie_predictions = pd.DataFrame(movie_similarities.dot(self.movie_features), index=self.movies_df.index, columns=self.movie_features.columns)

    def hybrid_filtering(self):
        self.hybrid_predictions = self.user_predictions.add(self.movie_predictions, fill_value=0)

    def recommend_movies(self, user_id, num_recommendations=10):
        user_ratings = self.ratings_df[self.ratings_df['userId'] == user_id]
        user_ratings = user_ratings.set_index('movieId')
        user_predictions = self.hybrid_predictions.loc[user_id].drop(user_ratings.index)
        recommended_movies = user_predictions.sort_values(ascending=False).head(num_recommendations)
        recommended_movies = recommended_movies.to_frame().reset_index()
        recommended_movies = recommended_movies.merge(self.movies_df, on='movieId')[['movieId', 'title', 'genres']]
        return recommended_movies

# Пример использования класса
movies_df = pd.read_csv('movies.csv')
ratings_df = pd.read_csv('ratings.csv')
recommender = HybridRecommender(movies_df, ratings_df)
recommender.preprocess_data()
recommender.collaborative_filtering()
recommender.content_filtering()
recommender.hybrid_filtering()
recommended_movies = recommender.recommend_movies(user_id=1)
print(recommended_movies)